In [0]:
%%sh

# ls -al /usr/lib/zeppelin/local-repo/helium-registry-cache/a6618b3e-540a-340e-b624-07bf7f2b5e7d
# ls -al /var/lib/zeppelin/local-repo/helium-registry-cache

ls -al /etc/zeppelin/conf.dist
cat /etc/zeppelin/conf.dist/zeppelin-site.xml








In [0]:
%%sh

aws s3 ls s3://b2c-prod-dca-bdp-data/BDP-PROD-APP-INT-QA/user_data/zeppelin_521/notebook/2G9J8782Y/note.json
aws s3 cp s3://b2c-prod-dca-bdp-data/BDP-PROD-APP-INT-QA/user_data/zeppelin_521/notebook/2G9J8782Y/note.json 
cat  /tmp/2G9J8782Y.json


In [0]:

print("Start")

In [0]:

from pyspark.storagelevel import StorageLevel
from pyspark.sql.functions import *
from pyspark.sql.types import BooleanType
from datetime import datetime


# Snowflake Production
sfOptions = {
  "sfURL" : "appannie_aa_int_prod.us-east-1.snowflakecomputing.com",
  "sfUser" : "app_bdp_data_validation_qa",
  "sfPassword" : "0HN#s@Wa5$1R8jVj",
  "sfDatabase" : "AA_INTELLIGENCE_PRODUCTION",
  "sfSchema" : "ADL_STORE_PAID",
  "sfWarehouse" : "wh_dod_read7"
}
SNOWFLAKE_SOURCE_NAME = "net.snowflake.spark.snowflake"


start_date = '2020-01-01'
end_date = '2021-01-30'

download_channel = spark.read.format(SNOWFLAKE_SOURCE_NAME) \
    .options(**sfOptions) \
    .option("query",  f"select distinct date from FACT_STORE_PRODUCT_DOWNLOAD_CHANNEL_V1_CLUSTER_BY_DATE where date>='{start_date}' and date<='{end_date}';") \
    .load()
download_attribution = spark.read.format(SNOWFLAKE_SOURCE_NAME) \
    .options(**sfOptions) \
    .option("query",  f"select distinct date from FACT_STORE_PRODUCT_DOWNLOAD_ATTRIBUTION_V1_CLUSTER_BY_DATE where date>='{start_date}' and date<='{end_date}';") \
    .load()

print(download_channel.count())
print(download_channel.count() == download_attribution.count())


In [0]:

from pyspark.storagelevel import StorageLevel
from pyspark.sql.functions import *
from datetime import date
import random


start = date(2020, 1, 1)
end = date(2021, 1, 31)

for i in random.sample(range(start.toordinal(), end.toordinal()), 20):
    current = str(date.fromordinal(i))
    
    download_channel = spark.read.format(SNOWFLAKE_SOURCE_NAME) \
        .options(**sfOptions) \
        .option("query",  f"select distinct(product_key) from FACT_STORE_PRODUCT_DOWNLOAD_CHANNEL_V1_CLUSTER_BY_DATE where 
        date='{current}';") \
        .load()
    download_attribution = spark.read.format(SNOWFLAKE_SOURCE_NAME) \
        .options(**sfOptions) \
        .option("query",  f"select distinct(product_key) from FACT_STORE_PRODUCT_DOWNLOAD_ATTRIBUTION_V1_CLUSTER_BY_DATE where date='{current}';") \
        .load()
    
    if download_channel.count() != download_attribution.count():
        print(f"Count dismatch on date: {current}")
        break
        
    print(".", end="")
    
print("Checked.")


In [0]:

from pyspark.storagelevel import StorageLevel
from pyspark.sql.functions import *
from datetime import date
import random


start = date(2020, 1, 1)
end = date(2021, 1, 31)

for i in random.sample(range(start.toordinal(), end.toordinal()), 20):
    current = str(date.fromordinal(i))
    
    download_channel = spark.read.format(SNOWFLAKE_SOURCE_NAME) \
        .options(**sfOptions) \
        .option("query",  f"select product_key from FACT_STORE_PRODUCT_DOWNLOAD_CHANNEL_V1_CLUSTER_BY_DATE where (device_code='ios-phone' or device_code='ios-tablet' or device_code='android-all') and date='{current}';") \
        .load()
    download_attribution = spark.read.format(SNOWFLAKE_SOURCE_NAME) \
        .options(**sfOptions) \
        .option("query",  f"select product_key from FACT_STORE_PRODUCT_DOWNLOAD_ATTRIBUTION_V1_CLUSTER_BY_DATE where (device_code='ios-phone' or device_code='ios-tablet' or device_code='android-all') and date='{current}';") \
        .load()
    
    if download_channel.count() != download_attribution.count():
        print(download_channel.count())
        print(download_attribution.count())
        print(f"Count dismatch on date: {current}")
        break
        
    print(".", end="")
    
print("Checked.")


In [0]:

from pyspark.storagelevel import StorageLevel
from pyspark.sql.functions import *
from datetime import date
import random

start = date(2020, 1, 1)
end = date(2021, 1, 31)

for i in random.sample(range(start.toordinal(), end.toordinal()), 10):
    current = str(date.fromordinal(i))
    
    dc = spark.read.format(SNOWFLAKE_SOURCE_NAME) \
        .options(**sfOptions) \
        .option("query",  f"select * from FACT_STORE_PRODUCT_DOWNLOAD_CHANNEL_V1_CLUSTER_BY_DATE where (device_code='ios-phone' or device_code='ios-tablet' or device_code='android-all') and date='{current}';") \
        .load()
    dc.persist(StorageLevel.MEMORY_ONLY)
    da = spark.read.format(SNOWFLAKE_SOURCE_NAME) \
        .options(**sfOptions) \
        .option("query",  f"select * from FACT_STORE_PRODUCT_DOWNLOAD_ATTRIBUTION_V1_CLUSTER_BY_DATE where (device_code='ios-phone' or device_code='ios-tablet' or device_code='android-all') and date='{current}';") \
        .load()
    da.persist(StorageLevel.MEMORY_ONLY)

    right = dc.selectExpr("device_key", "country_key", "product_key",
                          "EST_ORGANIC_DOWNLOAD as v1", "EST_PAID_DOWNLOAD as v2",
                          "EST_PAID_SEARCH_DOWNLOAD as v3", "EST_PAID_IN_APP_ADS_DOWNLOAD as v4", "EST_ORGANIC_SEARCH_DOWNLOAD as v5", "EST_ORGANIC_FEATURED_DOWNLOAD as v6",
                          "EST_DOWNLOAD as t")
    left = da.selectExpr("device_key", "country_key", "product_key", 
                         "EST_ORGANIC_DOWNLOAD as old_v1", "EST_PAID_DOWNLOAD as old_v2")
    joined = left.join(right, ["device_key", "country_key", "product_key"], how='left')
    
    if joined.filter("v1!=old_v1 or v2!=old_v2 or v1!=v5+v6 or v2!=v3+v4").count() != 0:
        print(f"Data dismatch on date: {current}")
        break
        
    print(".", end="")
    
print("Checked.")


In [0]:


download_channel = spark.read.format(SNOWFLAKE_SOURCE_NAME) \
    .options(**sfOptions) \
    .option("query",  f"select * from FACT_STORE_PRODUCT_DOWNLOAD_CHANNEL_V1_CLUSTER_BY_DATE where  (device_code='ios-phone' or device_code='ios-tablet' or device_code='android-all') and date>='{start_date}' and date<='{end_date}';") \
    .load()
download_attribution = spark.read.format(SNOWFLAKE_SOURCE_NAME) \
    .options(**sfOptions) \
    .option("query",  f"select * from FACT_STORE_PRODUCT_DOWNLOAD_ATTRIBUTION_V1_CLUSTER_BY_DATE where  (device_code='ios-phone' or device_code='ios-tablet' or device_code='android-all') and date>='{start_date}' and date<='{end_date}';") \
    .load()

left = download_channel.selectExpr("sum(est_download) as t", "sum(est_organic_download) as v1", "sum(est_paid_download) as v2",
                                   "sum(est_paid_search_download) as v3", 
                                   "sum(est_paid_in_app_ads_download) as v4", 
                                   "sum(est_organic_search_download) as v5", 
                                   "sum(est_organic_featured_download) as v6").collect()
right = download_attribution.selectExpr("sum(est_organic_download) as old_v1",
                                        "sum(est_paid_download) as old_v2").collect()
    
print(left[0].t == left[0].v1+left[0].v2)
print(left[0].t == left[0].v3+left[0].v4+left[0].v5+left[0].v6)

print(left[0].v1 == right[0].old_v1)
print(left[0].v2 == right[0].old_v2)

In [0]:

from pyspark.storagelevel import StorageLevel
from pyspark.sql.functions import *
from datetime import date
import random


start = date(2020, 1, 1)
end = date(2021, 1, 31)

for i in random.sample(range(start.toordinal(), end.toordinal()), 20):
    i = date(2021, 1, 10).toordinal()
    current = str(date.fromordinal(i))
    
    dc = spark.read.format(SNOWFLAKE_SOURCE_NAME) \
        .options(**sfOptions) \
        .option("query",  f"select sum(est_organic_download) as v1, sum(est_paid_download) as v2 from FACT_STORE_PRODUCT_DOWNLOAD_CHANNEL_V1_CLUSTER_BY_DATE where  (device_code='ios-phone' or device_code='ios-tablet' or device_code='android-all') and date='{current}';") \
        .load().collect()
    da = spark.read.format(SNOWFLAKE_SOURCE_NAME) \
        .options(**sfOptions) \
        .option("query",  f"select sum(est_organic_download) as v1, sum(est_paid_download) as v2 from FACT_STORE_PRODUCT_DOWNLOAD_ATTRIBUTION_V1_CLUSTER_BY_DATE where  (device_code='ios-phone' or device_code='ios-tablet' or device_code='android-all') and date='{current}';") \
        .load().collect()
    
    if dc[0].V1 != da[0].V1 or dc[0].V2 != da[0].V2:
        print(dc)
        print(da)
        print(f"Count dismatch on date: {current}")
        break
        
    print(".", end="")


print("Checked.")